In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


def GetData(fileHandle):
    data = fileHandle.readline()
    data = data[:len(data)-1].split(" ") #split the data
    data = [int(d) for d in data]  #change data to number
    return data

def IsStartOfPacket(data):
    return data[0] == 85 and data[1] == 0 and data[2] - data[5] == 3




In [ ]:
from math import floor

upsampleFactor = 8
# pulse_length = 461

def handleFile(f,f_pos):
    #read position
    data = f_pos.readline()
    pos_vec = []
    while(data):
        pos_vec.append(int(data))
        data = f_pos.readline()

    # %%
    frame_flag = 0 # this flag indicate if there is another message that should be
                # concatinated
    raw_compelete = 0 # this flag indicate if there is a compelete raw cir message
    offset = 0 #this is the offset of the cir from the start, now they are the same
    cir_compact = [] #just for show
    shift_compact = [] #just for show
    cir_ups_ml_compact = []
    cir_ups_ml_pahse_compact = []
    cir_ml_pahse_compact = []
    phase_compact = []
    count = 0
    pos_count = 0 # note how many line read from file handle f, increment when f is read
    event_pos = []

    img_compact = []
    real_compact = []
    cir_2_raw = {}

    try:
        while count != 1000000:
            #print(count)
            log = GetData(f)
            pos_count = pos_count+1
            if pos_count in pos_vec:
                event_pos.append(count)
            if IsStartOfPacket(log) and log[2] == 254 and log[3] == 1:
                length = log[5]-3
                frame_flag = 1
                nMessage = math.ceil((length-6)/7)+1
                cir_raw_1 = list()
                for i in range(1,nMessage+1):
                    log = GetData(f)
                    pos_count = pos_count+1
                    if pos_count in pos_vec:
                        event_pos.append(count)
                    if not log[0] == i:
                        break
                    cir_raw_1.extend(log[1:])
                cir_raw_1 = cir_raw_1[1:length+1]
            if frame_flag == 1 and log[2] == 158 and log[3] == 1:
                length = log[5]-3
                frame_flag = 0
                nMessage = math.ceil((length-6)/7)+1
                cir_raw_2 = list()
                for i in range(1,nMessage+1):
                    log = GetData(f)
                    pos_count = pos_count+1
                    if pos_count in pos_vec:
                        event_pos.append(count)
                    if not log[0] == i:
                        break
                    cir_raw_2.extend(log[1:])
                cir_raw_2 = cir_raw_2[1:length+1]
                cir_raw_1.extend(cir_raw_2)
                raw_compelete = 1
            if raw_compelete:
                raw_compelete = 0
                cir_raw = cir_raw_1
                data_rea = []
                data_img = []
                if len(cir_raw) != 400:
                    continue
                for j in range(0, len(cir_raw), 4):
                    dec = cir_raw[j+1]*256 + cir_raw[j]
                    if dec > 2**15-1:
                        dec = 2**16-dec
                        dec = 0-dec
                    data_rea.append(dec)
                for j in range(2, len(cir_raw), 4):
                    dec = cir_raw[j+1]*256 + cir_raw[j]
                    if dec > 2**15-1:
                        dec = 2**16-dec
                        dec = 0-dec
                    data_img.append(dec)

                cir = np.sqrt(np.array(data_rea)**2 + np.array(data_img)**2)
                img_compact.append(np.array(data_img))
                real_compact.append(np.array(data_rea))
                count = count+1
                cir_2_raw[count] = pos_count
        return cir_ups_ml_compact,cir_ups_ml_pahse_compact, cir_ml_pahse_compact, shift_compact, cir_compact, phase_compact, event_pos, img_compact, real_compact,cir_2_raw
    except:
        return cir_ups_ml_compact,cir_ups_ml_pahse_compact, cir_ml_pahse_compact, shift_compact, cir_compact, phase_compact, event_pos, img_compact, real_compact,cir_2_raw


# kick

In [ ]:
root_dir = "/home/bobxu/Documents/data/kick"   
item = "kick_xhb3600s5.csv"
f = open(root_dir+"/raw_data_demo/cir/"+item)
print("raw_data_demo/cir/"+item)
f_pos = open(root_dir+"/raw_data_demo/pos/log_"+item)
mag_comp, pahse_comp_up, pahse_comp_noups, shift_comp, cir_comp,phase_comp, event_pos,img_compact,real_compact,cir_2_raw  = handleFile(f,f_pos)


freq_s_ratio = 64 # frequency sampling ratio
N = len(real_compact[0])
real_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
img_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
for i in range(len(img_compact)):

    y = real_compact[i] + 1j*img_compact[i]
    Y = np.fft.fft(y)
    Y_pad = np.concatenate((Y[:N//2], np.zeros(N*(freq_s_ratio-1)), Y[N//2:]))

    # Transform back to time domain with higher sample rate
    y_pad = np.fft.ifft(Y_pad)

    real_compact_up[i] = y_pad.real
    img_compact_up[i] = y_pad.imag

i = 2
index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))

angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])



complex_val = real_compact_up[i] + 1j*img_compact_up[i]
complex_val = complex_val * np.exp(-1j*angle)

for i in range(len(img_compact)):
    index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))
    angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    complex_val = complex_val * np.exp(-1j*angle)

    real_compact_up[i] = complex_val.real
    img_compact_up[i] = complex_val.imag

cir_sinc_comp_len = 11
data_length = len(real_compact_up)
var_comp_c = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))
var_comp_b = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))

complex_val = real_compact_up[0] + 1j*img_compact_up[0]
index = np.argmax(abs(complex_val))
h = (complex_val)[index-freq_s_ratio*1:index + freq_s_ratio*10]
h_var_b = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001
h_var_c = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001

alpha_var_c = 0.1#0.5  # current variance update ratio
alpha_var_b = 0.002#0.01  # backgroud variance update ratio

for i in range(1,data_length):
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    index = np.argmax(abs(complex_val))
    
    if index >=64:
        complex_val = complex_val[index-freq_s_ratio*1:index + freq_s_ratio*10]
    else:
        complex_val = np.concatenate((complex_val[index-freq_s_ratio*1:],complex_val[:index + freq_s_ratio*10]))
    
    

    y = h - complex_val
    h = h - 0.1 * y
    h_var_b = h_var_b + alpha_var_b * (abs(y) - h_var_b)
    h_var_c = h_var_c + alpha_var_c * (abs(y) - h_var_c)
    var_comp_c[i] = h_var_c
    var_comp_b[i] = h_var_b



In [ ]:
# Given data
start_index = 3500
length = 200
freq_s_ratio = 64  

# Create the figure and plot
plt.figure(dpi=200)
# Assuming 'var_comp_c' and 'var_comp_b' are already defined elsewhere
img = plt.imshow((var_comp_c - 1.3*var_comp_b)[start_index:start_index+length, :int(1*freq_s_ratio/0.3)].T, vmax=6)  # Limit y-axis to 1 meter in data
# plt.colorbar(img, location="bottom")

# Modify x-axis to represent time with fewer ticks (every 3.2 seconds)
x_ticks = np.arange(0, length+1, 40)  # 20 samples = 1 second
x_labels = [f"{tick*0.05:.2f}" for tick in x_ticks]  # Convert sample to seconds
plt.xticks(x_ticks, x_labels)
plt.xlabel('time (s)')

# Modify y-axis to represent distance with fewer ticks (every 0.3 meters, only from 0 to 1 meter)
y_ticks = np.arange(0, int(1*freq_s_ratio/0.3)+1, freq_s_ratio/0.3/5)  # Convert 0.3 meters to items
y_labels = [f"{tick*0.3/freq_s_ratio:.1f}" for tick in y_ticks]  # Convert items to meters
plt.yticks(y_ticks, y_labels)
plt.ylabel('range (m)')

# Assuming 'event_pos' is already defined elsewhere
# for item in event_pos:
#     pos = item - start_index
#     x = [pos, pos]
#     y = [0, int(1*freq_s_ratio/0.3)]  # Limit y to 1 meter in the plotted line as well
#     if pos < 0:
#         continue
#     if pos > length:
#         continue
#     print(x)
#     plt.plot(x, y, color='red')  # Highlight the line in red (or any other color you prefer)
plt.savefig('../../figure/plotted/kick/kick.png')
plt.show()


# empty

In [ ]:
root_dir = "/home/bobxu/Documents/data/kick"   
item = "0208_empty_demo_test1.csv"
f = open(root_dir+"/raw_data_demo/cir/"+item)
print("raw_data_demo/cir/"+item)
f_pos = open(root_dir+"/raw_data_demo/pos/log_"+item)
mag_comp, pahse_comp_up, pahse_comp_noups, shift_comp, cir_comp,phase_comp, event_pos,img_compact,real_compact,cir_2_raw  = handleFile(f,f_pos)


freq_s_ratio = 64 # frequency sampling ratio
N = len(real_compact[0])
real_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
img_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
for i in range(len(img_compact)):

    y = real_compact[i] + 1j*img_compact[i]
    Y = np.fft.fft(y)
    Y_pad = np.concatenate((Y[:N//2], np.zeros(N*(freq_s_ratio-1)), Y[N//2:]))

    # Transform back to time domain with higher sample rate
    y_pad = np.fft.ifft(Y_pad)

    real_compact_up[i] = y_pad.real
    img_compact_up[i] = y_pad.imag

i = 2
index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))

angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])



complex_val = real_compact_up[i] + 1j*img_compact_up[i]
complex_val = complex_val * np.exp(-1j*angle)

for i in range(len(img_compact)):
    index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))
    angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    complex_val = complex_val * np.exp(-1j*angle)

    real_compact_up[i] = complex_val.real
    img_compact_up[i] = complex_val.imag

cir_sinc_comp_len = 11
data_length = len(real_compact_up)
var_comp_c = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))
var_comp_b = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))

complex_val = real_compact_up[0] + 1j*img_compact_up[0]
index = np.argmax(abs(complex_val))
h = (complex_val)[index-freq_s_ratio*1:index + freq_s_ratio*10]
h_var_b = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001
h_var_c = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001

alpha_var_c = 0.1#0.5  # current variance update ratio
alpha_var_b = 0.002#0.01  # backgroud variance update ratio

for i in range(1,data_length):
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    index = np.argmax(abs(complex_val))
    
    if index >=64:
        complex_val = complex_val[index-freq_s_ratio*1:index + freq_s_ratio*10]
    else:
        complex_val = np.concatenate((complex_val[index-freq_s_ratio*1:],complex_val[:index + freq_s_ratio*10]))
    
    

    y = h - complex_val
    h = h - 0.1 * y
    h_var_b = h_var_b + alpha_var_b * (abs(y) - h_var_b)
    h_var_c = h_var_c + alpha_var_c * (abs(y) - h_var_c)
    var_comp_c[i] = h_var_c
    var_comp_b[i] = h_var_b



In [ ]:
# Given data
start_index = 3500
length = 200
freq_s_ratio = 64  

# Create the figure and plot
plt.figure(dpi=200)
# Assuming 'var_comp_c' and 'var_comp_b' are already defined elsewhere
img = plt.imshow((var_comp_c - 1.3*var_comp_b)[start_index:start_index+length, :int(1*freq_s_ratio/0.3)].T, vmax=6)  # Limit y-axis to 1 meter in data
# plt.colorbar(img, location="bottom")

# Modify x-axis to represent time with fewer ticks (every 3.2 seconds)
x_ticks = np.arange(0, length+1, 40)  # 20 samples = 1 second
x_labels = [f"{tick*0.05:.2f}" for tick in x_ticks]  # Convert sample to seconds
plt.xticks(x_ticks, x_labels)
plt.xlabel('time (s)')

# Modify y-axis to represent distance with fewer ticks (every 0.3 meters, only from 0 to 1 meter)
y_ticks = np.arange(0, int(1*freq_s_ratio/0.3)+1, freq_s_ratio/0.3/5)  # Convert 0.3 meters to items
y_labels = [f"{tick*0.3/freq_s_ratio:.1f}" for tick in y_ticks]  # Convert items to meters
plt.yticks(y_ticks, y_labels)
plt.ylabel('range (m)')

# Assuming 'event_pos' is already defined elsewhere
# for item in event_pos:
#     pos = item - start_index
#     x = [pos, pos]
#     y = [0, int(1*freq_s_ratio/0.3)]  # Limit y to 1 meter in the plotted line as well
#     if pos < 0:
#         continue
#     if pos > length:
#         continue
#     print(x)
#     plt.plot(x, y, color='red')  # Highlight the line in red (or any other color you prefer)
plt.savefig('../../figure/plotted/kick/empty.png')
plt.show()


# walk

In [ ]:
root_dir = "/home/bobxu/Documents/data/kick"   
item = "0208_walk_test1.csv"
f = open(root_dir+"/raw_data_demo/cir/"+item)
print("raw_data_demo/cir/"+item)
f_pos = open(root_dir+"/raw_data_demo/pos/log_"+item)
mag_comp, pahse_comp_up, pahse_comp_noups, shift_comp, cir_comp,phase_comp, event_pos,img_compact,real_compact,cir_2_raw  = handleFile(f,f_pos)


freq_s_ratio = 64 # frequency sampling ratio
N = len(real_compact[0])
real_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
img_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
for i in range(len(img_compact)):

    y = real_compact[i] + 1j*img_compact[i]
    Y = np.fft.fft(y)
    Y_pad = np.concatenate((Y[:N//2], np.zeros(N*(freq_s_ratio-1)), Y[N//2:]))

    # Transform back to time domain with higher sample rate
    y_pad = np.fft.ifft(Y_pad)

    real_compact_up[i] = y_pad.real
    img_compact_up[i] = y_pad.imag

i = 2
index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))

angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])



complex_val = real_compact_up[i] + 1j*img_compact_up[i]
complex_val = complex_val * np.exp(-1j*angle)

for i in range(len(img_compact)):
    index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))
    angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    complex_val = complex_val * np.exp(-1j*angle)

    real_compact_up[i] = complex_val.real
    img_compact_up[i] = complex_val.imag

cir_sinc_comp_len = 11
data_length = len(real_compact_up)
var_comp_c = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))
var_comp_b = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))

complex_val = real_compact_up[0] + 1j*img_compact_up[0]
index = np.argmax(abs(complex_val))
h = (complex_val)[index-freq_s_ratio*1:index + freq_s_ratio*10]
h_var_b = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001
h_var_c = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001

alpha_var_c = 0.1#0.5  # current variance update ratio
alpha_var_b = 0.002#0.01  # backgroud variance update ratio

for i in range(1,data_length):
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    index = np.argmax(abs(complex_val))
    
    if index >=64:
        complex_val = complex_val[index-freq_s_ratio*1:index + freq_s_ratio*10]
    else:
        complex_val = np.concatenate((complex_val[index-freq_s_ratio*1:],complex_val[:index + freq_s_ratio*10]))
    
    

    y = h - complex_val
    h = h - 0.1 * y
    h_var_b = h_var_b + alpha_var_b * (abs(y) - h_var_b)
    h_var_c = h_var_c + alpha_var_c * (abs(y) - h_var_c)
    var_comp_c[i] = h_var_c
    var_comp_b[i] = h_var_b



In [ ]:
# Given data
start_index = 1500
length = 200
freq_s_ratio = 64  

# Create the figure and plot
plt.figure(dpi=200)
# Assuming 'var_comp_c' and 'var_comp_b' are already defined elsewhere
img = plt.imshow((var_comp_c - 1.3*var_comp_b)[start_index:start_index+length, :int(1*freq_s_ratio/0.3)].T, vmax=6)  # Limit y-axis to 1 meter in data
# plt.colorbar(img, location="bottom")

# Modify x-axis to represent time with fewer ticks (every 3.2 seconds)
x_ticks = np.arange(0, length+1, 40)  # 20 samples = 1 second
x_labels = [f"{tick*0.05:.2f}" for tick in x_ticks]  # Convert sample to seconds
plt.xticks(x_ticks, x_labels)
plt.xlabel('time (s)')

# Modify y-axis to represent distance with fewer ticks (every 0.3 meters, only from 0 to 1 meter)
y_ticks = np.arange(0, int(1*freq_s_ratio/0.3)+1, freq_s_ratio/0.3/5)  # Convert 0.3 meters to items
y_labels = [f"{tick*0.3/freq_s_ratio:.1f}" for tick in y_ticks]  # Convert items to meters
plt.yticks(y_ticks, y_labels)
plt.ylabel('range (m)')

# Assuming 'event_pos' is already defined elsewhere
# for item in event_pos:
#     pos = item - start_index
#     x = [pos, pos]
#     y = [0, int(1*freq_s_ratio/0.3)]  # Limit y to 1 meter in the plotted line as well
#     if pos < 0:
#         continue
#     if pos > length:
#         continue
#     print(x)
#     plt.plot(x, y, color='red')  # Highlight the line in red (or any other color you prefer)
plt.savefig('../../figure/plotted/kick/walk.png')
plt.show()


# stand

In [ ]:
root_dir = "/home/bobxu/Documents/data/kick"   
item = "stand_xhb1060s0.csv"
f = open(root_dir+"/raw_data_demo/cir/"+item)
print("raw_data_demo/cir/"+item)
f_pos = open(root_dir+"/raw_data_demo/pos/log_"+item)
mag_comp, pahse_comp_up, pahse_comp_noups, shift_comp, cir_comp,phase_comp, event_pos,img_compact,real_compact,cir_2_raw  = handleFile(f,f_pos)


freq_s_ratio = 64 # frequency sampling ratio
N = len(real_compact[0])
real_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
img_compact_up = np.zeros((len(img_compact),100*freq_s_ratio))
for i in range(len(img_compact)):

    y = real_compact[i] + 1j*img_compact[i]
    Y = np.fft.fft(y)
    Y_pad = np.concatenate((Y[:N//2], np.zeros(N*(freq_s_ratio-1)), Y[N//2:]))

    # Transform back to time domain with higher sample rate
    y_pad = np.fft.ifft(Y_pad)

    real_compact_up[i] = y_pad.real
    img_compact_up[i] = y_pad.imag

i = 2
index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))

angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])



complex_val = real_compact_up[i] + 1j*img_compact_up[i]
complex_val = complex_val * np.exp(-1j*angle)

for i in range(len(img_compact)):
    index = np.argmax(abs(real_compact_up[i] + 1j*img_compact_up[i]))
    angle = np.angle(real_compact_up[i][index]+1j*img_compact_up[i][index])
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    complex_val = complex_val * np.exp(-1j*angle)

    real_compact_up[i] = complex_val.real
    img_compact_up[i] = complex_val.imag

cir_sinc_comp_len = 11
data_length = len(real_compact_up)
var_comp_c = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))
var_comp_b = np.zeros((data_length,cir_sinc_comp_len*freq_s_ratio))

complex_val = real_compact_up[0] + 1j*img_compact_up[0]
index = np.argmax(abs(complex_val))
h = (complex_val)[index-freq_s_ratio*1:index + freq_s_ratio*10]
h_var_b = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001
h_var_c = np.ones(cir_sinc_comp_len*freq_s_ratio)*0.001

alpha_var_c = 0.1#0.5  # current variance update ratio
alpha_var_b = 0.002#0.01  # backgroud variance update ratio

for i in range(1,data_length):
    complex_val = real_compact_up[i] + 1j*img_compact_up[i]
    index = np.argmax(abs(complex_val))
    
    if index >=64:
        complex_val = complex_val[index-freq_s_ratio*1:index + freq_s_ratio*10]
    else:
        complex_val = np.concatenate((complex_val[index-freq_s_ratio*1:],complex_val[:index + freq_s_ratio*10]))
    
    

    y = h - complex_val
    h = h - 0.1 * y
    h_var_b = h_var_b + alpha_var_b * (abs(y) - h_var_b)
    h_var_c = h_var_c + alpha_var_c * (abs(y) - h_var_c)
    var_comp_c[i] = h_var_c
    var_comp_b[i] = h_var_b



In [ ]:
# Given data
start_index = 4500+192-66
length = 200
freq_s_ratio = 64  

# Create the figure and plot
plt.figure(dpi=200)
# Assuming 'var_comp_c' and 'var_comp_b' are already defined elsewhere
img = plt.imshow((var_comp_c - 1.3*var_comp_b)[start_index:start_index+length, :int(1*freq_s_ratio/0.3)].T, vmax=6)  # Limit y-axis to 1 meter in data
# plt.colorbar(img, location="bottom")

# Modify x-axis to represent time with fewer ticks (every 3.2 seconds)
x_ticks = np.arange(0, length+1, 40)  # 20 samples = 1 second
x_labels = [f"{tick*0.05:.2f}" for tick in x_ticks]  # Convert sample to seconds
plt.xticks(x_ticks, x_labels)
plt.xlabel('time (s)')

# Modify y-axis to represent distance with fewer ticks (every 0.3 meters, only from 0 to 1 meter)
y_ticks = np.arange(0, int(1*freq_s_ratio/0.3)+1, freq_s_ratio/0.3/5)  # Convert 0.3 meters to items
y_labels = [f"{tick*0.3/freq_s_ratio:.1f}" for tick in y_ticks]  # Convert items to meters
plt.yticks(y_ticks, y_labels)
plt.ylabel('range (m)')

# Assuming 'event_pos' is already defined elsewhere
# for item in event_pos:
#     pos = item - start_index
#     x = [pos, pos]
#     y = [0, int(1*freq_s_ratio/0.3)]  # Limit y to 1 meter in the plotted line as well
#     if pos < 0:
#         continue
#     if pos > length:
#         continue
#     print(x)
#     plt.plot(x, y, color='red')  # Highlight the line in red (or any other color you prefer)
plt.savefig('../../figure/plotted/kick/stand.png')
plt.show()
